# Lista de Exercícios 01

## Quebrando Shift Cipher

Utilização da lógica modular

Considera a diferença de código ASCII dos caracteres maiúsculos e minúsculos

Mantém a pontuação, mas não considera caracteres especiais (é,ã,Ç, etc.)

In [83]:
def shift_cipher(plain_text: str, k=3):
    letters = []
    for c in plain_text:
        letter_ascii = ord(c)
        if letter_ascii in range(65, 91):
            offset = 65

        if letter_ascii in range(95, 123):
            offset = 97

        letter_code = letter_ascii - offset
        if letter_code in range(26):
            shifted_letter = chr((letter_code + k) % 26 + offset)
            letters.append(shifted_letter)

        else:
            letters.append(c)

    cipher_text = "".join(letters)
    return cipher_text


plain_text_example = "Duas coisas são infinitas: o universo e a estupidez humana. Mas, em relação ao universo, ainda não tenho certeza absoluta."
shift_cipher(plain_text_example, 3)

'Gxdv frlvdv vãr lqilqlwdv: r xqlyhuvr h d hvwxslghc kxpdqd. Pdv, hp uhodçãr dr xqlyhuvr, dlqgd qãr whqkr fhuwhcd devroxwd.'

Reutiliza o algoritmo de cifra, mas com o negativo da chave
Desloca de volta

In [84]:
def shift_decipher(cipher_text: str, k=3):
    plain_text = shift_cipher(cipher_text,-k)
    return plain_text

cipher_text_example = "Tkqi seyiqi iãe ydvydyjqi: e kdyluhie u q uijkfytup xkcqdq. Cqi, uc hubqçãe qe kdyluhie, qydtq dãe judxe suhjupq qriebkjq."
shift_decipher(cipher_text_example, 16)

'Duas coisas são infinitas: o universo e a estupidez humana. Mas, em relação ao universo, ainda não tenho certeza absoluta.'

### Quebra por força bruta

In [85]:
import sqlite3


def get_frequent_words():
    db_path = "combinacoes.db"
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT palavra FROM palavras_comuns")
    
    return [p[0] for p in cursor.fetchall()]

get_frequent_words()

['o',
 'a',
 'e',
 'de',
 'que',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'entre',
 'era',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'esse',
 'está',
 'estão',
 'ser',
 'tem',
 'foi',
 'ter',
 'me',
 'há',
 'nos',
 'meu',
 'minha',
 'bem',
 'tudo',
 'você',
 'talvez']

Função rudimentar para checar se o resultado é em texto claro inteligível

In [86]:
def not_gibberish(text):
    common_words = get_frequent_words()
    words = text.lower().split()

    words_found = sum([common_words.count(word) for word in words])
    
    return words_found > 3


not_gibberish(
    "Sim, eu quero queijo. Como você sabia? Talvez você consiga ler mentes?!!"
)

True

In [87]:
def break_shift_cipher_v1(cipher_text: str):
    for k in range(1,26):
        plain_text = shift_decipher(cipher_text,k)
        if not_gibberish(plain_text):
            return k
    return None


break_shift_cipher_v1(cipher_text_example)

16

### Quebra por distribuição frequência

In [88]:
def get_letter_frequency():
    db_path = "combinacoes.db"
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(
        "SELECT combinacao, porcentagem FROM combinacoes ORDER BY porcentagem DESC"
    )

    # Criar dicionários separados por tamanho da chave
    freq_1 = {}  # Para combinações de 1 letra
    freq_2 = {}  # Para combinações de 2 letras
    freq_3 = {}  # Para combinações de 3 letras

    # Separar os dados nos dicionários apropriados
    for combinacao, porcentagem in cursor.fetchall():
        if len(combinacao) == 1:
            freq_1[combinacao] = porcentagem
        elif len(combinacao) == 2:
            freq_2[combinacao] = porcentagem
        elif len(combinacao) == 3:
            freq_3[combinacao] = porcentagem

    conn.close()

    return freq_1, freq_2, freq_3  # Retorna os três dicionários separados



In [89]:
def find_best_matches(cipher_letter_frequency):
    freq_1, freq_2, freq_3 = get_letter_frequency()
    cfreq_1, cfreq_2, cfreq_3 = {}, {}, {}

    # Ordena as frequências do texto cifrado do maior para o menor
    cipher_letter_frequency_ordered = dict(
        sorted(cipher_letter_frequency.items(), key=lambda item: item[1], reverse=True)
    )

    for combinacao, porcentagem in cipher_letter_frequency_ordered.items():
        if len(combinacao) == 1:
            cfreq_1[combinacao] = porcentagem
        elif len(combinacao) == 2:
            cfreq_2[combinacao] = porcentagem
        elif len(combinacao) == 3:
            cfreq_3[combinacao] = porcentagem

    matches = {}
    matches.update(dict(zip(cfreq_1, freq_1)))
    matches.update(dict(zip(cfreq_2, freq_2)))
    matches.update(dict(zip(cfreq_3, freq_3)))

    return matches

In [90]:
import itertools

def break_shift_cipher_v2(cipher_text: str):
    alphabet = [chr(i) for i in range(ord("a"), ord("z") + 1)]
    digraphs = ["".join(pair) for pair in itertools.product(alphabet, repeat=2)]
    trigraphs = ["".join(triplet) for triplet in itertools.product(alphabet, repeat=3)]

    total_letters = sum(1 for c in cipher_text if c.isalpha())

    cipher_letter_frequency = {}
    for letter in alphabet + digraphs + trigraphs:
        cipher_letter_frequency[letter] = round(
            cipher_text.count(letter) / total_letters, 2
        )
    
    matches = find_best_matches(cipher_letter_frequency)

    for k, v in matches.items():
        key = abs(ord(k) - ord(v))
        clear_text = shift_decipher(cipher_text, key)
        if not_gibberish(clear_text):
            break

    return shift_decipher(cipher_text, key)


break_shift_cipher_v2(cipher_text_example)

'Duas coisas são infinitas: o universo e a estupidez humana. Mas, em relação ao universo, ainda não tenho certeza absoluta.'

## Quebrando Cifra por Transposição

In [91]:
def columnar_transposition_cipher(plain_text: str, key: str):
    columns = [[] for _ in range(5)]

    key_order = key
    for c, i in zip(list(sorted(key)), range(len(key))):
        key_order = key_order.replace(c, str(i + 1))

    padding = (5 - len(plain_text) % 5) % 5
    plain_text += "qwerty"[:padding]

    for i in range(len(plain_text)):
        columns[i % 5].append(plain_text[i])

    shuffled_columns = []
    for n in list(map(int, key_order)):
        shuffled_columns.append(columns[n - 1])

    shuffled_letters = []
    for i in range(len(plain_text)):
        shuffled_letters.append(shuffled_columns[i % 5][i // 5])

    cipher_text = "".join(shuffled_letters)
    return cipher_text


columnar_transposition_cipher(plain_text_example, "cargo")

'uD asocais sosãi infinsta :uo inrveos  e atespueid zmhuna a.aM s,mee raloçãa uo inrveosa, ni daãnto ne hoecertazb aostlu.aeqw'

In [92]:
def columnar_transposition_decipher(cipher_text: str, key: str):
    columns = [[] for _ in range(5)]

    key_order = key
    for c, i in zip(list(sorted(key)), range(len(key))):
        key_order = key_order.replace(c, str(i + 1))

    for i in range(len(cipher_text)):
        columns[i % 5].append(cipher_text[i])

    unshuffled_columns = []
    for i in range(1,6):
        unshuffled_columns.append(columns[key_order.index(str(i))])

    unshuffled_letters = []
    for i in range(len(cipher_text)):
        unshuffled_letters.append(unshuffled_columns[i % 5][i // 5])

    cipher_text = "".join(unshuffled_letters)
    return cipher_text

cipher_text_example2 = "uD asocais sosãi infinsta :uo inrveos  e atespueid zmhuna a.aM s,mee raloçãa uo inrveosa, ni daãnto ne hoecertazb aostlu.aeqw"
columnar_transposition_decipher(cipher_text_example2, "cargo")

'Duas coisas são infinitas: o universo e a estupidez humana. Mas, em relação ao universo, ainda não tenho certeza absoluta.qwe'

### Quebra por força bruta

Como eu só tenho 5 colunas que são organizadas conforme a ordem das letras da palavra, temos 5! possibilidades de embaralhamento que podem ser geradas por diversas combinações de palavras.
Por exemplo, usamos "cargo" para codificar a mensagem anterior, o que no algoritmo atribui os valores "21534"
o qual também pode ser obtido pela chave "baecd". Logo, só é necessário achar uma permutação de "abcde" que gere a sequência de ordenação correta para encontrarmos uma chave de decriptação válida.

In [93]:
import string

def break_columnar_transposition_cipher_v1(cipher_text: str):
    key_combinations = ["".join(p) for p in itertools.permutations("abcde")]
    for key in key_combinations:
        plain_text = columnar_transposition_decipher(cipher_text, key)
        if not_gibberish(plain_text):
            return key
    return None


key = break_columnar_transposition_cipher_v1(cipher_text_example2)
columnar_transposition_decipher(cipher_text_example2, key)

'Duas coisas são infinitas: o universo e a estupidez humana. Mas, em relação ao universo, ainda não tenho certeza absoluta.qwe'

### Quebra por distribuição frequência

In [94]:
def break_columnar_transposition_cipher_v2(cipher_text: str):
    return